# Precipitation Data Preprocessing

This Jupyter Notebook is dedicated to the initial preprocessing of the historical precipitation data retrieved from NOAA FTP servers, transforming raw records into a clean and temporally consistent format suitable for subsequent hydrological analysis. The primary objective is to ensure the data's quality and integrity before its application in hydrological modeling.

The preprocessing steps undertaken in this notebook will primarily focus on **data cleaning and quality control**. This involves systematically addressing missing values,  and resolving inconsistencies within the 15-minute precipitation records to ensure the reliability of the input data. Identifying and correcting any detected outliers or erroneous readings will be performed on another notebook after the storm events have been selected. Furthermore, emphasis will be placed on ensuring **temporal consistency and alignment** across all records, verifying that the time series conform to a uniform resolution and are properly synchronized for accurate hydrological computations. The output of this notebook will be a refined dataset of quality-controlled and temporally consistent precipitation data, prepared for further processing such as storm event identification and ultimate use in driving the ModClark model.

Documentation for NOAA precipitation data is available in the following link: https://www1.ncdc.noaa.gov/pub/data/hpd/auto/v2/beta/15min/readme.csv.txt

In [ ]:
# Import modules
import pandas as pd
import geopandas as gpd
from pathlib import Path
import warnings
import sys
import numpy as np
from datetime import datetime


# Supress warnings
warnings.filterwarnings("ignore")

In [2]:
# Import created functions
project_root_path = Path.cwd().parent.parent
if str(project_root_path) not in sys.path:
    sys.path.append(str(project_root_path))
from src.utils.data_utils import glimpse


### Load and Inspect Precipitation Data

In [3]:
# Build list of files
ppt_folder_path = project_root_path / 'data/bronze/tabular/precipitation'
ppt_file_list_path = list(ppt_folder_path.glob('*.csv'))
ppt_example_path = ppt_file_list_path[0]

In [4]:
# Inspect example file
ppt_example_df = pd.read_csv(ppt_example_path)
glimpse(ppt_example_df)

Rows: 10765
Columns: 491
           Null Count  Dtype    First Values
           ----------  -----    -------------
STATION    0           object   [USC00463669, USC00463669, USC00463669, USC00463669, USC00463669]
LATITUDE   0           float64  [38.4311, 38.4311, 38.4311, 38.4311, 38.4311]
LONGITUDE  0           float64  [-79.819, -79.819, -79.819, -79.819, -79.819]
ELEVATION  0           float64  [836.7, 836.7, 836.7, 836.7, 836.7]
DATE       0           object   [1992-06-02, 1992-06-03, 1992-06-04, 1992-06-05, 1992-06-06]
ELEMENT    0           object   [QPCP, QPCP, QPCP, QPCP, QPCP]
0000Val    0           int64    [-9999, 0, 0, 0, 0]
0000MF     0           object   [ , Z, Z, Z, Z]
0000QF     0           object   [M,  ,  ,  ,  ]
0000S1     0           object   [6, 6, 6, 6, 6]
0000S2     0           object   [ ,  ,  ,  ,  ]
0015Val    0           int64    [-9999, 0, 0, 0, 0]
0015MF     0           object   [ , Z, Z, Z, Z]
0015QF     0           object   [M,  ,  ,  ,  ]
0015S1     0  

The precipitation data is in wide-format. Each row represents a daily observation. The meaning of columns extracted from  documentation are given below:

- **HR00Val:** Is the value on the first hour of the day (i.e., the precipitation
           total during the time of day 00:00-01:00; missing = -9999).
           The units are _**hundredths of inch**_.
- **HR0MF**: is the measurement flag for the first hour of the day.
- **HR00QF**: is the quality flag for the first hour of the day.
- **HR00S1**: is the primary source flag for the first hour of the day.
- **HR00S2**: is the secondary source flag for the second hour of the day.

... and so on through the 24th hour of the day.

- **DlySum**: is the sum of all the non-missing, non-quality-control-flagged
           hourly values in the day (i.e., daily sum).
- **DlySumMF**: is the measurement flag placeholder that applies to the daily sum
           for consistency with the hourly value measurement flag.
           Currently this is always blank.
- **DlySumQF**: is the quality flag for the daily sum. It is blank if all 24 hourly
           values in the day were used to compute the sum,i
           and set to "P" (for Partial daily sum) if fewer than all 24 hourly
           values were used.
- **DlySumS1**: is the primary source flag for the daily sum for consistency with the
           hourly value primary source flag. Currently this is always blank.
- **DlySumS2**: is the secondary source flag for the daily sum for consistency with
           the hourly value secondary source flag. It is always set to "C" to
           indicate the daily sum is Computed from the hourly values preceding it in the same daily data record ... as opposed to a daily sum that might be reported by an observer who examines a daily accumulation amount in a rain gauge. 

In [5]:
print('Sample unique values for measurement flags:')
np.unique(np.concatenate([ppt_example_df['0000MF'].unique(), ppt_example_df['0015MF'].unique(), ppt_example_df['0030MF'].unique()]))

Sample unique values for measurement flags:


array([' ', '.', 'Z', 'a', 'g'], dtype=object)

The flag meanings, also extracted from the documentation, are as follow:

Blank = no measurement information applicable
- **g** = a carry-over measurement flag from the DSI-3240 dataset
                   which was used only on the very first hour of the month
                   if there was zero precipitation during that hour. The purpose
                   of this flag was mainly to indicate that the station was
                   functional and reporting during the month. Normally in
                   DSI-3240, **zero precipitation amounts were not included in the
                   data file in order to save space**. This HPD dataset does
                   include zero precipitation totals, both those assumed from
                   the DSI-3240 dataset and those determined from the digital
                   recordings of bucket level data. 
- **Z** = represents an "assumed" zero precipitation total. Usually
                   these are values from the DSI-3240 dataset. The rule in that
                   dataset was to "assume" a zero total for any hour where
                   nothing else was reported or indicated for that hour as long
                   as the very first hour of the month had a non-zero amount or
                   a zero amount with the "g" measurement flag. Zero amounts
                   were omitted from the DSI-3240 dataset in order to save disk
                   space. We are not concerned with that anymore.
- **a** = represents the beginning hour of an accumulation period.
                   Sometimes multi-hour accumulations were reported in DSI-3240,
                   so they were brought over for the period of record in which
                   we have DSI-3240 data when that was the best information
                   available. The data value for the beginning of an
                   accumulation period is set to missing (-9999).
- **.** = represents an hour during an accumulation period, between the
                   beginning and ending hours of the accumulation period.
                   The data value during the midst of an accumulation period is
                   set to missing (-9999). Accumulations across multiple hours
                   exist only from the legacy DSI-3240 data source. 
- **A** = designates the end of an accumulation period.
                   The accumulation total for the period is given for the
                   data value.
- **T** = trace of precipitation


In [6]:
print('Sample unique values for quality flags:')
np.unique(np.concatenate([ppt_example_df['0000QF'].unique(), ppt_example_df['0015QF'].unique(), ppt_example_df['0030QF'].unique()]))

Sample unique values for quality flags:


array([' ', 'D', 'M', 'R'], dtype=object)

Meanings for the quality flag, also extracted from documentation, are as follow:

- **Blank** = did not fail any quality assurance check
- **X**     = failed global extreme exceedence check
- **N**     = failed negative precipitation check
- **Y**     = failed state extreme exceedence check
        (performed on daily totals)
- **K**     = failed streak/frequent-value check
- **G**     = failed gap check
- **O**     = failed climatological outlier check
- **Z**     = flagged as a result of an official Datzilla investigation
- **A**     = The value is not an hourly precipitation total but rather
        an accumulation total for a period greater than an hour in
        duration and lasting through the end of this hour.
        (See measurement flags "a" and "." for the beginning time of
        the accumulation period and times during the accumulation
        period, respectively.) Accumulations across multiple hours
        exist only from the legacy DSI-3240 data source.
- **M**     = represents the associated value at this observation time is
        missing in the DSI-3240 dataset and no alternate data source
        is available. This is a carry-over indicator from DSI-3240 to
        allow the user to distinguish between missing and deleted
        data in that older system. (See the "D" quality flag.)
        However, the most consistent way to identify hours of missing
        data across the entire dataset is to test if the
        precipitation value is equal to the special missing value
        of -9999. 
- **D**     = represents the associated value at this time was deleted by
        the DSI-3240 processing system. Usually this was done
        manually by a trained meteorological technician who made the
        decision using ancillary information and experience.
- **Q**     = a carry-over quality flag the legacy DSI-3240 data source. 
        Prior to 1996: Indicates value failed an extreme value test
        (value will be present); data are to be used with caution.
        Extremes tests were:
        1) If the value was not an accumulated precipitation total,
            the value failed the one-hour statewide 100-year return
            period precipitation.
        2) If the value was an accumulated precipitation total,
            the value failed the 24-hour statewide extreme
            precipitation total. This flag was assigned during a 1997
            NCDC rehabilitation of the 1900-1995 DSI-3240 archive.
        Since January 1996: A single erroneous datum (value will be
        present). Lowest data resolution is hourly. This flag is
        rarely used in DSI-3240 since 1996.
- **q**     = a carry-over quality flag the legacy DSI-3240 data source. 
        Used since January 1996. An hourly value excludes one or
        more 15 minute periods. Lowest data resolution is 15 minutes.
- **R**     = a carry-over quality flag the legacy DSI-3240 data source.
        Used since January 1996. Indicates data values are suspect
        with regard to the times or period of occurrence. 


The following steps are taken to transform the raw tabular data, into a format suitable to initiate analysis:

1. Drops unnecessary columns: `['STATION', 'ELEMENT','LATITUDE', 'LONGITUDE', 'ELEVATION', 'DlySum', 'DlySumMF', 'DlySumQF', 'DlySumS1', 'DlySumS2']`.
2. Melts the DataFrame to long format based on the 'DATE' column.
3. Combines 'DATE' and 'Time_Attribute' for a new timestamp.
4. Extracts 'Attribute' from the 'Time_Attribute' column.
5. Pivots the data to separate out different measurements.
6. Renames the columns for clarity.
7. Replace -9999 (null values) by NA
8. Replace height values for failed quality cheks by NA
9. Filter out observation prior to year 2014 and after 2025-05-08, if any.
10. Fill in with NA values missing intervals

In [7]:
df = ppt_example_df.copy()
columns_to_trop = ['STATION', 'ELEMENT','LATITUDE', 'LONGITUDE', 'ELEVATION', 'DlySum', 'DlySumMF', 'DlySumQF', 'DlySumS1', 'DlySumS2']
df = df.drop(columns=columns_to_trop)

# Identifying columns to melt
id_vars = ['DATE']
value_vars = [col for col in df.columns if col not in id_vars]

# Melting to long format
df_long = df.melt(id_vars=id_vars,
                    value_vars=value_vars,
                    var_name='Time_Attribute',
                    value_name='Value')

# Add Time to DATE column and extract Attribute from 'Time_Attribute'
df_long['DATE'] = df_long['DATE'].astype(str) + ' ' + df_long['Time_Attribute'].str[:4]
df_long['Attribute'] = df_long['Time_Attribute'].str[4:]

df_long = df_long.drop(columns=['Time_Attribute'])

In [8]:
df_long.head()

,DATE,Value,Attribute
0,1992-06-02 0000,-9999,Val
1,1992-06-03 0000,0,Val
2,1992-06-04 0000,0,Val
3,1992-06-05 0000,0,Val
4,1992-06-06 0000,0,Val


In [9]:
df_long['Attribute'].unique()

array(['Val', 'MF', 'QF', 'S1', 'S2'], dtype=object)

In [10]:
df_long['Value'].unique()

array([-9999, 0, 10, 20, 30, 12, 1, 2, 4, 9, 3, 7, 5, 6, 13, 14, 8, ' ',
       'Z', 'g', '.', 'a', 'M', 'D', 'R', '6', 'H', 40, 21, 27, 19, 17,
       11, 16, 24, 26, 50, 15, 'A', 18, 49, 22, 60, 37, 'N', 29, 23, 34,
       25, 80, 70, 28, 116, 117, 35, 'Q', 114, 'q', 44, 68, 63, 115, 86,
       110, 38, 90, 100, 55, 41, 32, 57, 45, 58, 75, 33, 48, 42, 47, 43,
       51, 'G', 130, 36, 61, 31, 39], dtype=object)

In [11]:
# Pivoting to separate Value, Measurement Flag, Quality Flag, and Source Flags
df_final = df_long.pivot_table(
    index=['DATE'], 
    columns='Attribute', 
    values='Value', 
    aggfunc='first'
).reset_index()


In [12]:
df_final.head()

Attribute,DATE,MF,QF,S1,S2,Val
0,1992-06-02 0000,,M,6,,-9999
1,1992-06-02 0015,,M,6,,-9999
2,1992-06-02 0030,,M,6,,-9999
3,1992-06-02 0045,,M,6,,-9999
4,1992-06-02 0100,,M,6,,-9999


In [13]:
df_final['MF'].unique()

array([' ', 'Z', 'g', 'a', '.', 'A'], dtype=object)

In [14]:
df_final['QF'].unique()

array(['M', ' ', 'A', 'D', 'R', 'G', 'Z', 'Q', 'q', 'N'], dtype=object)

In [15]:
# Rename  Columns
df_final.columns.name = None 
df_final = df_final.rename({
    'DATE':'date',
    'Val':'height',
    'MF':'measurement_flag',
    'QF':'quality_flag',
}, axis='columns')

# Drop columns
drop_columns = ['S1','S2']
df_final = df_final.drop(columns=drop_columns)

# Parse datetime column
df_final['date'] = pd.to_datetime(df_final['date'], format='%Y-%m-%d %H%M')

# Replace null values
df_final['height'] = df_final['height'].replace(-9999,pd.NA)

# Filter by date
cutoff_date = pd.to_datetime('20140101')
df_final = df_final[df_final['date']>=cutoff_date]

In [16]:
df_final.head()

,date,measurement_flag,quality_flag,height
655200,2014-01-01 00:00:00,,,0
655201,2014-01-01 00:15:00,,,0
655202,2014-01-01 00:30:00,,,0
655203,2014-01-01 00:45:00,,,0
655204,2014-01-01 01:00:00,,,0


In [17]:
df_final['measurement_flag'].unique()

array([' '], dtype=object)

In [18]:
df_final['quality_flag'].unique()

array([' ', 'N'], dtype=object)

In [19]:
glimpse(df_final)

Rows: 378240
Columns: 4
                  Null Count  Dtype           First Values
                  ----------  -----           -------------
date              0           datetime64[ns]  [2014-01-01T00:00:00.000000000, 2014-01-01T00:15:00.000000000, 2014-01-01T00:30:00.000000000, 2014-01-01T00:45:00.000000000, 2014-01-01T01:00:00.000000000]
measurement_flag  0           object          [ ,  ,  ,  ,  ]
quality_flag      0           object          [ ,  ,  ,  ,  ]
height            914         object          [0, 0, 0, 0, 0]


In [20]:
print(f'Sum of negative precipitation values: {sum(df_final['height']<0)}')

Sum of negative precipitation values: 0


In [21]:
df_final['height'] = df_final['height'].where(df_final['height'] >= 0, pd.NA)
df_final['height'].unique()

array([0, 1, 2, 5, 3, 4, 7, 6, 16, <NA>, 8, 12, 29, 10, 11, 9, 18, 15, 13,
       14, 19, 50, 20, 32, 75, 33, 27, 37, 47, 44, 17, 21, 63, 24, 49, 22,
       28, 31, 23, 36, 38, 26, 25, 39, 60, 30, 58, 57, 45, 35, 117, 86,
       68, 48, 42, 51, 43, 116, 114, 115, 61, 40, 55, 41], dtype=object)

In [22]:
df_final['height'].value_counts()

height
0      360522
1        8758
2        3312
3        1710
4         995
        ...  
115         1
61          1
40          1
55          1
41          1
Name: count, Length: 63, dtype: int64

In [23]:
df_final['measurement_flag'].value_counts()

measurement_flag
    378240
Name: count, dtype: int64

In [24]:
df_final['quality_flag'].value_counts()

quality_flag
     378190
N        50
Name: count, dtype: int64

In [25]:
df_final[df_final['quality_flag']=='N']

,date,measurement_flag,quality_flag,height
659472,2014-02-14 12:00:00,,N,0
659554,2014-03-06 08:30:00,,N,0
659555,2014-03-06 08:45:00,,N,0
773125,2017-08-03 09:15:00,,N,0
773126,2017-08-03 09:30:00,,N,0
773127,2017-08-03 09:45:00,,N,0
775901,2017-09-01 07:15:00,,N,0
775902,2017-09-01 07:30:00,,N,0
775903,2017-09-01 07:45:00,,N,0
777232,2017-09-15 04:00:00,,N,0


#### Negative precipitation check

The quality flag "N" indicates that negative precipitation check failed. Therefore, for those measurement, NA will be attributed for the height value.

In [26]:
# Replace failed to check negative precipitation with NA
df_final['quality_flag'] = df_final['quality_flag'].replace('N', pd.NA)

In [27]:
df_final.describe()

,date
count,378240
mean,2019-09-04 18:34:09.898476288
min,2014-01-01 00:00:00
25%,2016-12-01 23:56:15
50%,2019-08-17 23:52:30
75%,2022-07-08 23:48:45
max,2025-04-01 23:45:00


##### Check if there are missing interval

In [28]:
def count_15min_intervals(date_min, date_max):
    if isinstance(date_min, str):
        date_min = datetime.strptime(date_min, '%Y%m%d %H:%M:%S')
    if isinstance(date_max, str):
        date_max = datetime.strptime(date_max, '%Y%m%d %H:%M:%S')
    delta = date_max - date_min
    total_minutes = delta.total_seconds() / 60
    return int(total_minutes // 15) + 1

In [29]:
missing_interval = count_15min_intervals(df_final['date'].min(), df_final['date'].max()) - df_final.shape[0]
print(f'Missing 15 minute intervals: {missing_interval}')

Missing 15 minute intervals: 16224


For this data example, there are over **16224** of missing interval. That's about half an year. Further  investigation will be perfomed to check the possiblity of zero precipitation data have been ommitted to save space. 

In [30]:
# Define function to generate 15 minute sequence
def generate_15min_sequence(date_min, date_max):
    if isinstance(date_min, str):
        date_min = datetime.strptime(date_min, '%Y%m%d %H:%M:%S')
    if isinstance(date_max, str):
        date_max = datetime.strptime(date_max, '%Y%m%d %H:%M:%S')
    
    return pd.date_range(start=date_min, end=date_max, freq='15min')


In [31]:
time_seq = generate_15min_sequence(df_final['date'].min(), df_final['date'].max())
filled_seq_df = pd.DataFrame({'seq':time_seq}).merge(
    df_final,
    how='left',
    left_on='seq',
    right_on='date'
)

filled_seq_df['year'] = filled_seq_df['seq'].dt.year
missing_by_year = filled_seq_df.groupby('year')['date'].apply(lambda x: x.isna().sum())
print('Number of missing values by year:')
print(missing_by_year)

Number of missing values by year:
year
2014    7584
2015       0
2016     192
2017      96
2018       0
2019     288
2020     192
2021    3552
2022    3072
2023       0
2024       0
2025    1248
Name: date, dtype: int64


In [32]:
count_zeros_height_by_year = df_final.assign(year= df_final['date'].dt.year).groupby('year')['height'].apply(lambda x: sum(x==0))
print(f'Count of zero height values by year: {count_zeros_height_by_year}')

Count of zero height values by year: year
2014    26060
2015    33347
2016    33422
2017    33320
2018    32875
2019    33173
2020    33031
2021    30217
2022    30505
2023    33742
2024    33734
2025     7096
Name: height, dtype: int64


In [33]:
filled_seq_df.head()

,seq,date,measurement_flag,quality_flag,height,year
0,2014-01-01 00:00:00,2014-01-01 00:00:00,,,0,2014
1,2014-01-01 00:15:00,2014-01-01 00:15:00,,,0,2014
2,2014-01-01 00:30:00,2014-01-01 00:30:00,,,0,2014
3,2014-01-01 00:45:00,2014-01-01 00:45:00,,,0,2014
4,2014-01-01 01:00:00,2014-01-01 01:00:00,,,0,2014


If the missing values, consistently was after an specific year, and zeros for data was missing for that specific year, than it was possible zeros was ommitted from data. However, that's not what is observed from data, as shown above. Therefore, the procedure is to fill those missing dates with NA for height values.

In [34]:
filled_seq_df = filled_seq_df.drop(columns=['date', 'year'])
filled_seq_df = filled_seq_df.rename(columns={'seq':'date'})
df_final = filled_seq_df.copy()
glimpse(df_final)

Rows: 394464
Columns: 4
                  Null Count  Dtype           First Values
                  ----------  -----           -------------
date              0           datetime64[ns]  [2014-01-01T00:00:00.000000000, 2014-01-01T00:15:00.000000000, 2014-01-01T00:30:00.000000000, 2014-01-01T00:45:00.000000000, 2014-01-01T01:00:00.000000000]
measurement_flag  16224       object          [ ,  ,  ,  ,  ]
quality_flag      16274       object          [ ,  ,  ,  ,  ]
height            17138       object          [0, 0, 0, 0, 0]


In [35]:
print(f'Missing 15min interval:{count_15min_intervals(df_final['date'].min(), df_final['date'].max()) - df_final.shape[0]}')

Missing 15min interval:0


#### Change Measure Unit

From the documentation, measure unit are **hundredths of inch**. Therefore, to convert to inch precipitation values must be devided by 100. The code below convert precipitation values to **inch**. 

In [36]:
df_final['height'] = df_final['height']/100
df_final['height'].unique()

array([0.0, 0.01, 0.02, 0.05, 0.03, 0.04, 0.07, 0.06, 0.16, <NA>, nan,
       0.08, 0.12, 0.29, 0.1, 0.11, 0.09, 0.18, 0.15, 0.13, 0.14, 0.19,
       0.5, 0.2, 0.32, 0.75, 0.33, 0.27, 0.37, 0.47, 0.44, 0.17, 0.21,
       0.63, 0.24, 0.49, 0.22, 0.28, 0.31, 0.23, 0.36, 0.38, 0.26, 0.25,
       0.39, 0.6, 0.3, 0.58, 0.57, 0.45, 0.35, 1.17, 0.86, 0.68, 0.48,
       0.42, 0.51, 0.43, 1.16, 1.14, 1.15, 0.61, 0.4, 0.55, 0.41],
      dtype=object)

In [37]:
glimpse(df_final)

Rows: 394464
Columns: 4
                  Null Count  Dtype           First Values
                  ----------  -----           -------------
date              0           datetime64[ns]  [2014-01-01T00:00:00.000000000, 2014-01-01T00:15:00.000000000, 2014-01-01T00:30:00.000000000, 2014-01-01T00:45:00.000000000, 2014-01-01T01:00:00.000000000]
measurement_flag  16224       object          [ ,  ,  ,  ,  ]
quality_flag      16274       object          [ ,  ,  ,  ,  ]
height            17138       object          [0.0, 0.0, 0.0, 0.0, 0.0]


In [38]:
# Convert height dtype to float
df_final['height'] = pd.to_numeric(df_final['height'], errors='coerce')
df_final['height'] = df_final['height'].astype(float)
glimpse(df_final)

Rows: 394464
Columns: 4
                  Null Count  Dtype           First Values
                  ----------  -----           -------------
date              0           datetime64[ns]  [2014-01-01T00:00:00.000000000, 2014-01-01T00:15:00.000000000, 2014-01-01T00:30:00.000000000, 2014-01-01T00:45:00.000000000, 2014-01-01T01:00:00.000000000]
measurement_flag  16224       object          [ ,  ,  ,  ,  ]
quality_flag      16274       object          [ ,  ,  ,  ,  ]
height            17138       float64         [0.0, 0.0, 0.0, 0.0, 0.0]


### Batch Clean Precipitation Data

All steps in the initial preprocessing of precipitation data are defined. This section, clean salve cleaned precipitation dataset, locally.

The previous steps in cleaning the data was wrapped into a function facilitate the cleaning process thorugh a loop.

In [4]:
# Import function to clean precipitation data
from src.preprocessing.clean_preciptation_tabular_data import clean_preciptation_data

In [41]:
# Test the function on an station example
ppt_example_df = pd.read_csv(ppt_example_path)
ppt_example_cleaned_df = clean_preciptation_data(ppt_example_df)
glimpse(ppt_example_cleaned_df)

Rows: 394464
Columns: 4
                  Null Count  Dtype           First Values
                  ----------  -----           -------------
date              0           datetime64[ns]  [2014-01-01T00:00:00.000000000, 2014-01-01T00:15:00.000000000, 2014-01-01T00:30:00.000000000, 2014-01-01T00:45:00.000000000, 2014-01-01T01:00:00.000000000]
measurement_flag  16224       object          [ ,  ,  ,  ,  ]
quality_flag      16274       object          [ ,  ,  ,  ,  ]
height            17138       float64         [0.0, 0.0, 0.0, 0.0, 0.0]


In [51]:
# Define the path to save the cleaned data
cleaned_ppt_data_folder_path = project_root_path / 'data/silver/tabular/precipitation'
cleaned_ppt_data_folder_path.mkdir(parents=True, exist_ok=True)

# Define function to save cleaned data
def save_cleaned_ppt_data(df, output_path):
    try:
        df.to_parquet(output_path, index=False)
    except Exception as e:
        print(f"Error saving cleaned data: {e}")

# Loop through each file in the list
count = 0
for ppt_path in ppt_file_list_path:
    count += 1
    list_length = len(ppt_file_list_path)
    print(f"Processing file {count}/{list_length}: {ppt_path}")

    # Define the cleaned file name and path
    cleaned_ppt_file_name = ppt_path.stem + '.parquet'
    cleaned_ppt_file_path = cleaned_ppt_data_folder_path / cleaned_ppt_file_name

    # Check if the cleaned file already exists
    if cleaned_ppt_file_path.exists():
        print(f"Cleaned file already exists: {cleaned_ppt_file_path}")
        continue

    # Read the raw data
    try:
        ppt_df = pd.read_csv(ppt_path)
    except Exception as e:
        print(f"Error reading file {ppt_path}: {e}")
        continue

    # Clean the data
    try:
        cleaned_ppt_df = clean_preciptation_data(ppt_df)
    except Exception as e:
        print(f"Error cleaning data from file {ppt_path}: {e}")
        continue

    # Save the cleaned data
    save_cleaned_ppt_data(cleaned_ppt_df, cleaned_ppt_file_path)
    print(f"Cleaned data saved to {cleaned_ppt_file_path}")

Processing file 1/260: /Users/alan/Data Science Projects/ML-ModClark-IUH-Model/data/bronze/tabular/precipitation/USC00463669.csv
Cleaned data saved to /Users/alan/Data Science Projects/ML-ModClark-IUH-Model/data/silver/tabular/precipitation/USC00463669.parquet
Processing file 2/260: /Users/alan/Data Science Projects/ML-ModClark-IUH-Model/data/bronze/tabular/precipitation/USC00338378.csv
Cleaned data saved to /Users/alan/Data Science Projects/ML-ModClark-IUH-Model/data/silver/tabular/precipitation/USC00338378.parquet
Processing file 3/260: /Users/alan/Data Science Projects/ML-ModClark-IUH-Model/data/bronze/tabular/precipitation/USW00014806.csv
Cleaned data saved to /Users/alan/Data Science Projects/ML-ModClark-IUH-Model/data/silver/tabular/precipitation/USW00014806.parquet
Processing file 4/260: /Users/alan/Data Science Projects/ML-ModClark-IUH-Model/data/bronze/tabular/precipitation/USC00407884.csv
Cleaned data saved to /Users/alan/Data Science Projects/ML-ModClark-IUH-Model/data/silve

**Cleaning Process Fail**

Some of data files, could not be cleaned. The code below investigates those data files.

In [5]:
# Identify failed precipitation files
raw_ppt_folder_path = project_root_path / 'data/bronze/tabular/precipitation'
cleaned_ppt_folder_path = project_root_path / 'data/silver/tabular/precipitation'

raw_ppt_file_path = raw_ppt_folder_path.glob('*.csv')
raw_ppt_file_names = [file.stem for file in raw_ppt_file_path]
cleaned_ppt_file_path = cleaned_ppt_folder_path.glob('*.parquet')
cleaned_ppt_file_names = [file.stem for file in cleaned_ppt_file_path]

failed_ppt_files_bool = [False if file in cleaned_ppt_file_names else True for file in raw_ppt_file_names]
failed_ppt_files_name = [file for file, bool in zip(raw_ppt_file_names, failed_ppt_files_bool) if bool == True]

print(f'Number of failed precipitationfiles: {len(failed_ppt_files_name)}')
print(f'Failed precipitation files: {format(failed_ppt_files_name)}')

Number of failed precipitationfiles: 15
Failed precipitation files: ['USC00332090', 'USC00465341', 'USC00116819', 'USC00469086', 'USC00466591', 'USC00468351', 'USC00303033', 'USC00465672', 'USC00468286', 'USC00448547', 'USC00449301', 'USC00469011', 'USC00153744', 'USC00401587', 'USC00335041']


In [6]:
failed_ppt_files_path = [raw_ppt_folder_path / (file + '.csv') for file in failed_ppt_files_name]
failed_ppt_files_path

[PosixPath('/Users/alan/Data Science Projects/ML-ModClark-IUH-Model/data/bronze/tabular/precipitation/USC00332090.csv'),
 PosixPath('/Users/alan/Data Science Projects/ML-ModClark-IUH-Model/data/bronze/tabular/precipitation/USC00465341.csv'),
 PosixPath('/Users/alan/Data Science Projects/ML-ModClark-IUH-Model/data/bronze/tabular/precipitation/USC00116819.csv'),
 PosixPath('/Users/alan/Data Science Projects/ML-ModClark-IUH-Model/data/bronze/tabular/precipitation/USC00469086.csv'),
 PosixPath('/Users/alan/Data Science Projects/ML-ModClark-IUH-Model/data/bronze/tabular/precipitation/USC00466591.csv'),
 PosixPath('/Users/alan/Data Science Projects/ML-ModClark-IUH-Model/data/bronze/tabular/precipitation/USC00468351.csv'),
 PosixPath('/Users/alan/Data Science Projects/ML-ModClark-IUH-Model/data/bronze/tabular/precipitation/USC00303033.csv'),
 PosixPath('/Users/alan/Data Science Projects/ML-ModClark-IUH-Model/data/bronze/tabular/precipitation/USC00465672.csv'),
 PosixPath('/Users/alan/Data Sci

In [7]:
# Load example failed precipitation file
failed_ppt_example = pd.read_csv(failed_ppt_files_path[0])
glimpse(failed_ppt_example)

Rows: 11
Columns: 491
           Null Count  Dtype    First Values
           ----------  -----    -------------
STATION    0           object   [USC00332090, USC00332090, USC00332090, USC00332090, USC00332090]
LATITUDE   0           float64  [39.6254, 39.6254, 39.6254, 39.6254, 39.6254]
LONGITUDE  0           float64  [-83.2132, -83.2132, -83.2132, -83.2132, -83.2132]
ELEVATION  0           float64  [262.1, 262.1, 262.1, 262.1, 262.1]
DATE       0           object   [2012-09-10, 2012-09-11, 2012-09-16, 2012-09-17, 2012-09-18]
ELEMENT    0           object   [QPCP, QPCP, QPCP, QPCP, QPCP]
0000Val    0           int64    [-9999, 1, -9999, 0, -9999]
0000MF     0           object   [ ,  ,  ,  ,  ]
0000QF     0           object   [ ,  ,  ,  ,  ]
0000S1     0           object   [H, H, H, H, H]
0000S2     0           object   [R, R, R, R, R]
0015Val    0           int64    [-9999, 0, -9999, 1, -9999]
0015MF     0           object   [ ,  ,  ,  ,  ]
0015QF     0           object   [ ,  ,  ,  ,

In [82]:
# Attempt to clean failed precipitation file
failed_ppt_example_cleaned = clean_preciptation_data(failed_ppt_example)

ValueError: Neither `start` nor `end` can be NaT

The issue is with the generating the time sequence. It is being passed null values to max/min date argument. 

In [11]:
failed_ppt_example_cleaned_non_fill = clean_preciptation_data(failed_ppt_example, False)
glimpse(failed_ppt_example_cleaned_non_fill)

Rows: 0
Columns: 4
                  Null Count  Dtype           First Values
                  ----------  -----           -------------
date              0           datetime64[ns]  []
measurement_flag  0           object          []
quality_flag      0           object          []
height            0           float64         []


In [14]:
failed_ppt_example_cleaned_non_fill.empty

True

The failed example data, is empty, after filtering by date. A thread of code was added to the original function to raise an err in case data is empty after filtering. The next code attempt to clean the previously failed data. 

In [10]:
# Define function to save cleaned data
def save_cleaned_ppt_data(df, output_path):
    try:
        df.to_parquet(output_path, index=False)
    except Exception as e:
        print(f"Error saving cleaned data: {e}")

cleaned_ppt_data_folder_path = project_root_path / 'data/silver/tabular/precipitation'
# Loop through each file in the list
count = 0
for ppt_path in failed_ppt_files_path:
    count += 1
    list_length = len(failed_ppt_files_path)
    print(f"Processing file {count}/{list_length}: {ppt_path}")

    # Define the cleaned file name and path
    cleaned_ppt_file_name = ppt_path.stem + '.parquet'
    cleaned_ppt_file_path = cleaned_ppt_data_folder_path / cleaned_ppt_file_name

    # Check if the cleaned file already exists
    if cleaned_ppt_file_path.exists():
        print(f"Cleaned file already exists: {cleaned_ppt_file_path}")
        continue

    # Read the raw data
    try:
        ppt_df = pd.read_csv(ppt_path)
    except Exception as e:
        print(f"Error reading file {ppt_path}: {e}")
        continue

    # Clean the data
    try:
        cleaned_ppt_df = clean_preciptation_data(ppt_df)
    except Exception as e:
        print(f"Error cleaning data from file {ppt_path}: {e}")
        continue

    # Save the cleaned data
    save_cleaned_ppt_data(cleaned_ppt_df, cleaned_ppt_file_path)
    print(f"Cleaned data saved to {cleaned_ppt_file_path}")

Processing file 1/15: /Users/alan/Data Science Projects/ML-ModClark-IUH-Model/data/bronze/tabular/precipitation/USC00332090.csv
Error cleaning data from file /Users/alan/Data Science Projects/ML-ModClark-IUH-Model/data/bronze/tabular/precipitation/USC00332090.csv: The DataFrame is empty after filtering by date. Please check the input data.
Processing file 2/15: /Users/alan/Data Science Projects/ML-ModClark-IUH-Model/data/bronze/tabular/precipitation/USC00465341.csv
Error cleaning data from file /Users/alan/Data Science Projects/ML-ModClark-IUH-Model/data/bronze/tabular/precipitation/USC00465341.csv: The DataFrame is empty after filtering by date. Please check the input data.
Processing file 3/15: /Users/alan/Data Science Projects/ML-ModClark-IUH-Model/data/bronze/tabular/precipitation/USC00116819.csv
Error cleaning data from file /Users/alan/Data Science Projects/ML-ModClark-IUH-Model/data/bronze/tabular/precipitation/USC00116819.csv: The DataFrame is empty after filtering by date. Ple

From above log, all failed data, does not have data covering period of study (2014 to 2025). Therefore those stations will be dropped from the modeling. 

In [11]:
print(f'Number of sucessfull cleaned precipitation files: {len(cleaned_ppt_file_names)}')

Number of sucessfull cleaned precipitation files: 245


#### Time  zone
From documentation it is  not clear if time for observations are in UTC are local time. It is assumed to be in UTC. However, assurance will be taken when exploring the storm events. If streamflow response to precipitation event has consistently, considerable lag, it possible the measurements are in local time. 

Per documentation, column `UTC_Offset`  "is the number of hours the station's local time is offset from GMT. Negative values earlier than GMT."

In [2]:
ppt_station_path = r'/Users/alan/Data Science Projects/ML-ModClark-IUH-Model/data/silver/geo/gpkg/study_area_ppt_stn.gpkg'
ppt_station = gpd.read_file(ppt_station_path)
ppt_station.head()

,index,StnID,Lat,Lon,Elev,State/Province,Name,WMO_ID,Sample_Interval (min),UTC_Offset,POR_Date_Range,PCT_POR_Good,Last_Half_POR,PCT_Last_Half_Good,Last_Qtr_POR,PCT_Last_Qtr_Good,geometry
0,11,USC00011099,34.9809,-85.8101,203.3,AL,BRIDGEPORT 5 NW,NaN,15,-6,19821101-20250109,87.2%,20031206-20250109,93.7%,20140623-20250109,96.2%,POINT (-85.8101 34.9809)
1,1540,USC00406162,35.2243,-85.8414,563.9,TN,MONTEAGLE,NaN,15,-6,19820601-20250108,89.8%,20030920-20250108,88.3%,20140515-20250108,94.9%,POINT (-85.8414 35.2243)
2,1535,USC00405187,35.4140,-86.8086,239.9,TN,LEWISBURG EXP STA,NaN,15,-6,19710501-20250117,81.8%,19980310-20250117,71.2%,20110814-20250117,61.8%,POINT (-86.8086 35.414)
3,1551,USC00408540,35.6763,-84.8547,230.1,TN,SPRING CITY,NaN,15,-5,20030501-20250106,92.8%,20140304-20250106,95.4%,20190805-20250106,90.9%,POINT (-84.8547 35.6763)
4,1525,USC00401587,35.7553,-87.4261,201.2,TN,CENTERVILLE WATER PL,NaN,15,-6,19820401-20060726,83.6%,19940529-20060726,75.8%,20000626-20060726,69.4%,POINT (-87.4261 35.7553)


In [3]:
ppt_station['UTC_Offset'].describe()

count    260.000000
mean      -5.250000
std        0.433848
min       -6.000000
25%       -5.250000
50%       -5.000000
75%       -5.000000
max       -5.000000
Name: UTC_Offset, dtype: float64